# Capacitated facility location problem

Uma companhia entrega diariamente seus produtos que são produzidos em 5 fábricas para 4 armazens. 

Nesse processo de entrega existem dois tipos de custos, o custo associado ao transporte e o custo fixo de uso da fábrica. 

Cada fábrica possui um limite quanto a quantidade de produtos a serem entregues. 

A companhia estuda fechar algumas fábricas para reduzir custos. 

Quais fábricas a companhia deve fechar, com a finalidade de reduzir os custos?

In [2]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [3]:
# Dados

# demanda dos armazéns
demand = [15, 18, 14, 20]

# capacidade de produção da cada fábrica
capacity = [20, 22, 17, 19, 18]

# custo fixo de cada fábrica
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# custo de transporte
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

In [4]:
# criando modelo
model = gp.Model("cfl")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-20


In [5]:
# y[p]: y[p] == 1 se a fábrica p é aberta, 0 caso contrário.
y = model.addVars(plants,vtype=GRB.BINARY,obj=fixedCosts,name="y")

In [6]:
# x[w,p]: quantidade transportada da fábrica p para o armazém i
x = model.addVars(warehouses, plants, obj=transCosts, name="x")

In [7]:
# função objetivo
model.modelSense = GRB.MINIMIZE
model.update()

In [8]:
# restrição de produção
model.addConstrs((x.sum('*', p) <= capacity[p] * y[p] for p in plants), "capacidade")
model.update()

In [9]:
# restrição de demanda
model.addConstrs((x.sum(w,'*') == demand[w] for w in warehouses), "demanda")
model.update()


In [10]:
# salva o modelo
model.write('cfacility_location.lp')

In [11]:
# usa o método barrier para resolver o relaxação linear na raiz
model.Params.method = 2


Set parameter Method to value 2


In [12]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0xb5a24dbe
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Found heuristic solution: objective 210900.00000
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl   

In [13]:
# imprime o valor ótimo
print(f'Custo total: {model.objVal}')


Custo total: 210500.0


In [14]:
print('Solução ótima:')
for p in plants:
    if y[p].x > 0.99:
        print(f'* Fábrica {p} é aberto')
        for w in warehouses:
            if x[w, p].x > 0:
                print(f'- Transporte de {x[w, p].x} unidades para o armazém {w}')
    else:
        print(f'* Fábrica {p} é fechada!')

Solução ótima:
* Fábrica 0 é aberto
- Transporte de 14.0 unidades para o armazém 2
- Transporte de 6.0 unidades para o armazém 3
* Fábrica 1 é aberto
- Transporte de 14.0 unidades para o armazém 0
- Transporte de 8.0 unidades para o armazém 3
* Fábrica 2 é fechada!
* Fábrica 3 é aberto
- Transporte de 1.0 unidades para o armazém 0
- Transporte de 18.0 unidades para o armazém 1
* Fábrica 4 é aberto
- Transporte de 6.0 unidades para o armazém 3
